In [1]:
import pennylane as qml
import os
import pennylane_snowflurry.utility.test_circuits as test_circuits
from pennylane_snowflurry.monarq_device import MonarqDevice
from pennylane_snowflurry.transpiler.transpiler_config import TranspilerConfig, MonarqDefaultConfig
from pennylane_snowflurry.transpiler.steps.base_decomposition import CliffordTDecomposition
from pennylane_snowflurry.transpiler.steps.routing import Swaps
from pennylane_snowflurry.transpiler.steps.optimization import IterativeCommuteAndMerge
from pennylane_snowflurry.transpiler.steps.native_decomposition import MonarqDecomposition
from pennylane_snowflurry.API.api_client import MonarqClient
from pennylane_snowflurry.API.api_adapter import ApiAdapter
from pennylane_snowflurry.utility.debug_utility import get_labels, arbitrary_circuit
from pennylane.tape import QuantumTape
from random import choice, random
from pennylane_snowflurry.transpiler.monarq_transpile import Transpiler

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


c:\Users\bouch\AppData\Local\Programs\Python\Python312\Lib\site-packages\pennylane\__init__.py:200: PennyLaneDeprecationWarning: Device will no longer be accessible top level. Please access the class as pennylane.devices.LegacyDevice
  warn(


In [2]:
client = MonarqClient("https://manager.anyonlabs.com", 
                      "stage",
                      "FjjIKjmDMoAMzSO4v2Bu62a+8vD39zib")

full_transpile = MonarqDefaultConfig(excluded_qubits=[9])
no_transpile = TranspilerConfig()
no_place = TranspilerConfig(CliffordTDecomposition(), Swaps(True, excluded_qubits=[9]), IterativeCommuteAndMerge(), MonarqDecomposition())
no_place_no_route = TranspilerConfig(CliffordTDecomposition(), IterativeCommuteAndMerge(), MonarqDecomposition())

In [3]:
# 6-ghz circuit
def ghz6():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.CNOT([0, 2])
    qml.CNOT([0, 3])
    qml.CNOT([0, 4])
    return qml.counts(wires=[0, 1, 2, 3, 4])

In [5]:
# get simulation and real result for 6-GHZ
sim_qnode = qml.QNode(ghz6, qml.device("default.qubit", shots=1000))
sim_results = sim_qnode()
sim_samples = []
for key in sim_results:
    count = sim_results[key]
    for _ in range(count):
        sim_samples.append(key)

print(sim_results)
monarq_dev = MonarqDevice(shots=1000, client=client, behaviour_config=no_transpile)
transpiler = Transpiler.get_transpiler(full_transpile)
bs_tape = transpiler(sim_qnode.tape)[0][0]

real_qnode = qml.QNode(lambda: arbitrary_circuit(bs_tape), monarq_dev)
real_results = real_qnode()
real_samples = []
for key in real_results:
    count = real_results[key]
    for _ in range(count):
        real_samples.append(key)
print(real_results)

{'00000': 504, '11111': 496}
{'00000': 239, '00001': 65, '00010': 61, '00011': 21, '00100': 62, '00101': 19, '00110': 14, '00111': 6, '01000': 52, '01001': 18, '01010': 20, '01011': 10, '01100': 23, '01101': 12, '01110': 11, '01111': 11, '10000': 19, '10001': 17, '10010': 11, '10011': 15, '10100': 16, '10101': 23, '10110': 11, '10111': 24, '11000': 18, '11001': 22, '11010': 22, '11011': 24, '11100': 23, '11101': 45, '11110': 20, '11111': 46}


In [ ]:
# get wires 
wires = [w for w in real_qnode.tape.wires]

print(wires)

[4, 5, 1, 8, 0]


In [46]:

real_samples = []
for key in real_results:
    count = real_results[key]
    for _ in range(count):
        real_samples.append(key)

# get noise model for each wire
noise_model : dict[int, float] = {w:0 for w in wires}
for i, w in enumerate(wires):
    for real in real_samples:
        one_count = sum([1 for r in real if r == '1'])
        if one_count > len(wires)/2 and real[i] == '1':
            noise_model[w] += 1
        elif one_count < len(wires)/2 and real[i] == '0':
            noise_model[w] += 1
    noise_model[w] /= 1000

print(noise_model)

{4: 0.851, 5: 0.768, 1: 0.749, 8: 0.771, 0: 0.759}


In [47]:
# create TREXes

ideal_dict = {"11111":500, "00000":500}
trex_samples = []
for key in ideal_dict:
    count = ideal_dict[key]
    for _ in range(count):
        trex = ""
        for i, w in enumerate(wires):
            if random() > noise_model[w]:
                trex += "0" if key[i] == "1" else "1"
            else:
                trex += key[i]
        trex_samples.append(trex)

trex_dict = {}
for sample in trex_samples:
    if sample not in trex_dict: trex_dict[sample] = 0
    trex_dict[sample] += 1

print(trex_dict)

{'11111': 134, '11001': 20, '11101': 42, '10111': 51, '10010': 11, '01011': 18, '11100': 15, '11011': 52, '11110': 54, '11010': 17, '01110': 14, '10101': 13, '00011': 14, '00100': 42, '01111': 28, '10100': 11, '10110': 18, '10011': 13, '00111': 14, '00110': 23, '01001': 18, '11000': 14, '00101': 14, '01101': 14, '00010': 45, '10001': 9, '01010': 14, '10000': 30, '01100': 22, '00000': 140, '01000': 32, '00001': 44}


In [48]:
# define circuit


def bernstein_vazirani(number : int, measurement = qml.counts):
    """bernstein vazirani for encoding given number
    """
    value = []
    while number > 0:
        value.insert(0, (number & 1) != 0)
        number = number >> 1

    num_wires = len(value) + 1
    [qml.Hadamard(wires[i]) for i in range(num_wires)]
    qml.Z(num_wires-1)
        
    # Uf
    [qml.CNOT([wires[i], wires[num_wires - 1]]) for i, should in enumerate(value) if should]
        
    [qml.Hadamard(wires[i]) for i in range(num_wires - 1)]
    return measurement(wires=[wires[i] for i in range(num_wires - 1)])
    

In [53]:
# make another 6 wires circuit
bs_func = lambda : bernstein_vazirani(11)
bs_qnode= qml.QNode(bs_func, qml.device("default.qubit", shots=1000))
bs_qnode()
bs_transpiler = Transpiler.get_transpiler(no_place)
bs_tape : QuantumTape = bs_transpiler(bs_qnode.tape)[0][0]
trexed_samples = []

for key in trex_dict:
    tape = type(bs_tape)(ops = bs_tape.operations, measurements=bs_tape.measurements, shots=trex_dict[key])
    dev = MonarqDevice(shots=trex_dict[key], client=client, behaviour_config=no_transpile)
    
    # add pauli x to trexted positions
    for i, w in enumerate(wires):
        if key[i] == '0': continue
        tape.operations.append(qml.PauliX(w))
    
    results = qml.execute([tape], device=dev)[0]
    
    # turn to samples
    result_samples = []
    for result in results:
        for _ in range(results[result]):
            result_samples.append(result)

    # add logic not to trexed positions
    for result in result_samples:
        trexed_result = ''
        for i, w in enumerate(wires):
            if key[i] == '0': 
                trexed_result += result[i]
                continue
            trexed_result += '1' if result[i] == '0' else '0'
            
        trexed_samples.append(trexed_result)
    
    # count trexes
    trexed_counts = {}
    for key in trexed_samples:
        if key not in trexed_counts:
            trexed_counts[key] = 0
        trexed_counts[key] += 1
    
print(trexed_counts)

IndexError: string index out of range